In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, roc_auc_score

import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import re

import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

c:\Users\vikra\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df=pd.read_csv(r"C:\Users\vikra\Documents\5153\mxmh_survey_results.csv")
df.columns

# Spotify feature extraction

In [9]:
genres = [
    ('Classical', 'https://open.spotify.com/playlist/37i9dQZF1DWWEJlAGA9gs0?si=e7d3dac97915451c'),
    ('Country', 'https://open.spotify.com/playlist/37i9dQZF1DX7aUUBCKwo4Y?si=5635901a93084f14'),
    ('EDM', 'https://open.spotify.com/playlist/37i9dQZF1DX3Kdv0IChEm9?si=8e964c6ffd4f42f4'),
    ('Folk', 'https://open.spotify.com/playlist/37i9dQZF1DWVmps5U8gHNv?si=18bce18f2da44226'),
    ('Gospel', 'https://open.spotify.com/playlist/37i9dQZF1DX7OIddoQVdRt?si=2bed0b648ba44069'),
    ('Hip hop', 'https://open.spotify.com/playlist/37i9dQZF1DXbkfWVLd8wE3?si=afcdd0d007fa46c3'),
    ('Jazz', 'https://open.spotify.com/playlist/37i9dQZF1DXbITWG1ZJKYt?si=guFyActQQw2vTGMobg6YSw&pi=a-aPfVZWhHRmKW'),
    ('K pop', 'https://open.spotify.com/playlist/37i9dQZF1DWTTXfIhcSkXG?si=1d8413b68211490f'),
    ('Latin', 'https://open.spotify.com/playlist/37i9dQZF1DX6ThddIjWuGT?si=ewnDCCmwQCmUuuW-OtcFQQ&pi=a-X8qfK7viSlOo'),
    ('Lofi', 'https://open.spotify.com/playlist/37i9dQZF1DX4t95PAs1EpY?si=kDgr4_IoRBeOioE6RM4arA&pi=a-0nKUKT8bSHmJ'),
    ('Metal', 'https://open.spotify.com/playlist/37i9dQZF1DWWOaP4H0w5b0?si=Izrft9RVRT2t8X595Ud8OA&pi=a-6cDsTZS3SJm8'),
    ('Pop', 'https://open.spotify.com/playlist/37i9dQZF1DX2vTOtsQ5Isl?si=a77c2bcb0ef24418'),
    ('R&B', 'https://open.spotify.com/playlist/37i9dQZF1DX4SBhb3fqCJd?si=eB_quwSsSbWxD4bUbE0PKw&pi=a-weeDuYyNSOyM'),
    ('Rap', 'https://open.spotify.com/playlist/2shZ4OmoeOmtAabLwHUiy7?si=W6gE09OZR7eYxPgbAW7cnw&pi=a-6DtXvfgxQzmY'),
    ('Rock', 'https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=w-lR2ajvRzmbmbMZmOAswA&pi=a-FoFG5A14Q6Og'),
    ('Video game music', 'https://open.spotify.com/playlist/4Pf4e2VIW38Lg5jNUnpWvg?si=0d93a6af0f954bc5')
]


In [43]:
cid='46fdb0714b7f41199db1c306d79168e4'
secret='cc44dadc45cc4c4a9b29d617db0c1602'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [10]:
def get_playlist_length(playlist_url):
    playlist_id = playlist_url.split('/')[-1].split('?')[0]
    playlist_info = sp.playlist(playlist_id)
    return len(playlist_info['tracks']['items'])


for genre, playlist_url in genres:
    try:
        playlist_length = get_playlist_length(playlist_url)
        print(f"Playlist '{genre}' has {playlist_length} tracks.")
    except Exception as e:
        print(f"Failed to retrieve playlist length for '{genre}': {e}")

Playlist 'Classical' has 100 tracks.
Playlist 'Country' has 50 tracks.
Playlist 'EDM' has 60 tracks.
Playlist 'Folk' has 100 tracks.
Playlist 'Gospel' has 88 tracks.
Playlist 'Hip hop' has 90 tracks.
Playlist 'Jazz' has 100 tracks.
Playlist 'K pop' has 40 tracks.
Playlist 'Latin' has 100 tracks.
Playlist 'Lofi' has 100 tracks.
Playlist 'Metal' has 100 tracks.
Playlist 'Pop' has 60 tracks.
Playlist 'R&B' has 50 tracks.
Playlist 'Rap' has 40 tracks.
Playlist 'Rock' has 100 tracks.
Playlist 'Video game music' has 94 tracks.


In [41]:
genres = [
    ('Lofi', 'https://open.spotify.com/playlist/37i9dQZF1DX4t95PAs1EpY?si=kDgr4_IoRBeOioE6RM4arA&pi=a-0nKUKT8bSHmJ'),
    ('Metal', 'https://open.spotify.com/playlist/37i9dQZF1DWWOaP4H0w5b0?si=Izrft9RVRT2t8X595Ud8OA&pi=a-6cDsTZS3SJm8'),
    ('Pop', 'https://open.spotify.com/playlist/37i9dQZF1DX2vTOtsQ5Isl?si=a77c2bcb0ef24418'),
    ('R&B', 'https://open.spotify.com/playlist/37i9dQZF1DX4SBhb3fqCJd?si=eB_quwSsSbWxD4bUbE0PKw&pi=a-weeDuYyNSOyM'),
    ('Rap', 'https://open.spotify.com/playlist/2shZ4OmoeOmtAabLwHUiy7?si=W6gE09OZR7eYxPgbAW7cnw&pi=a-6DtXvfgxQzmY'),
    ('Rock', 'https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=w-lR2ajvRzmbmbMZmOAswA&pi=a-FoFG5A14Q6Og'),
    ('Video game music', 'https://open.spotify.com/playlist/4Pf4e2VIW38Lg5jNUnpWvg?si=0d93a6af0f954bc5')
]


In [30]:
df_trial=df.copy()

In [33]:
df.to_csv(r"C:\Users\vikra\Documents\5153\spotifyGenresAfterLofi.csv")

In [44]:
df=pd.read_csv(r"C:\Users\vikra\Documents\5153\spotifyGenres.csv")

In [45]:
df.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0,0.2750,0.1570,7,-18.752,1,0.0636,0.890,0.842,0.186,0.3040,73.289,audio_features,17i5jLpzndlQhbS4SrTd0B,spotify:track:17i5jLpzndlQhbS4SrTd0B,https://api.spotify.com/v1/tracks/17i5jLpzndlQ...,https://api.spotify.com/v1/audio-analysis/17i5...,152280,4,Classical
1,1,0.2210,0.1260,0,-25.427,1,0.0447,0.989,0.897,0.102,0.2160,133.630,audio_features,4SFBV7SRNG2e2kyL1F6kjU,spotify:track:4SFBV7SRNG2e2kyL1F6kjU,https://api.spotify.com/v1/tracks/4SFBV7SRNG2e...,https://api.spotify.com/v1/audio-analysis/4SFB...,139307,4,Classical
2,2,0.2890,0.0306,9,-30.790,0,0.0446,0.987,0.911,0.102,0.1180,125.610,audio_features,3zLTPuucd3e6TxZnu2dlVS,spotify:track:3zLTPuucd3e6TxZnu2dlVS,https://api.spotify.com/v1/tracks/3zLTPuucd3e6...,https://api.spotify.com/v1/audio-analysis/3zLT...,212067,3,Classical
3,3,0.0753,0.0700,2,-27.272,1,0.0440,0.918,0.947,0.146,0.0625,79.801,audio_features,7h6GoPvGHC9uzZJ8bNvfIq,spotify:track:7h6GoPvGHC9uzZJ8bNvfIq,https://api.spotify.com/v1/tracks/7h6GoPvGHC9u...,https://api.spotify.com/v1/audio-analysis/7h6G...,365147,4,Classical
4,4,0.1300,0.1580,2,-16.132,1,0.0350,0.748,0.924,0.100,0.0998,85.031,audio_features,5x3TUfYzgZIdt0Kaop64ss,spotify:track:5x3TUfYzgZIdt0Kaop64ss,https://api.spotify.com/v1/tracks/5x3TUfYzgZId...,https://api.spotify.com/v1/audio-analysis/5x3T...,302093,4,Classical


In [46]:
def get_track_metadata(track_id):
    track_info = sp.track(track_id)
    song_name = track_info['name']
    artist_name = track_info['artists'][0]['name']  # Get the name of the first artist
    return song_name, artist_name

df['song_name'], df['artist_name'] = zip(*df['id'].apply(lambda x: get_track_metadata(x)))


In [47]:
df

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,artist_name
0,0,0.2750,0.1570,7,-18.752,1,0.0636,0.890000,0.842000,0.186,...,audio_features,17i5jLpzndlQhbS4SrTd0B,spotify:track:17i5jLpzndlQhbS4SrTd0B,https://api.spotify.com/v1/tracks/17i5jLpzndlQ...,https://api.spotify.com/v1/audio-analysis/17i5...,152280,4,Classical,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",Johann Sebastian Bach
1,1,0.2210,0.1260,0,-25.427,1,0.0447,0.989000,0.897000,0.102,...,audio_features,4SFBV7SRNG2e2kyL1F6kjU,spotify:track:4SFBV7SRNG2e2kyL1F6kjU,https://api.spotify.com/v1/tracks/4SFBV7SRNG2e...,https://api.spotify.com/v1/audio-analysis/4SFB...,139307,4,Classical,"The Well-Tempered Clavier, Book 1, BWV 846-869...",Johann Sebastian Bach
2,2,0.2890,0.0306,9,-30.790,0,0.0446,0.987000,0.911000,0.102,...,audio_features,3zLTPuucd3e6TxZnu2dlVS,spotify:track:3zLTPuucd3e6TxZnu2dlVS,https://api.spotify.com/v1/tracks/3zLTPuucd3e6...,https://api.spotify.com/v1/audio-analysis/3zLT...,212067,3,Classical,"Bagatelle No. 25 in A Minor, WoO 59 ""Für Elise""",Ludwig van Beethoven
3,3,0.0753,0.0700,2,-27.272,1,0.0440,0.918000,0.947000,0.146,...,audio_features,7h6GoPvGHC9uzZJ8bNvfIq,spotify:track:7h6GoPvGHC9uzZJ8bNvfIq,https://api.spotify.com/v1/tracks/7h6GoPvGHC9u...,https://api.spotify.com/v1/audio-analysis/7h6G...,365147,4,Classical,"Suite No. 3 in D Major, BWV 1068: II. Air",Johann Sebastian Bach
4,4,0.1300,0.1580,2,-16.132,1,0.0350,0.748000,0.924000,0.100,...,audio_features,5x3TUfYzgZIdt0Kaop64ss,spotify:track:5x3TUfYzgZIdt0Kaop64ss,https://api.spotify.com/v1/tracks/5x3TUfYzgZId...,https://api.spotify.com/v1/audio-analysis/5x3T...,302093,4,Classical,Pachelbel: Canon and Gigue in D Major for 3 Vi...,Johann Pachelbel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,1267,0.6530,0.5670,1,-9.355,1,0.1140,0.199000,0.000333,0.299,...,audio_features,62PN7n2p0bmGMiknhwo6Ga,spotify:track:62PN7n2p0bmGMiknhwo6Ga,https://api.spotify.com/v1/tracks/62PN7n2p0bmG...,https://api.spotify.com/v1/audio-analysis/62PN...,156823,5,Video game music,My Money,iSHi
1268,1268,0.6730,0.9590,1,-3.681,1,0.0875,0.003300,0.341000,0.237,...,audio_features,1Zcl82bovYTNH1MqUqrVDg,spotify:track:1Zcl82bovYTNH1MqUqrVDg,https://api.spotify.com/v1/tracks/1Zcl82bovYTN...,https://api.spotify.com/v1/audio-analysis/1Zcl...,143479,4,Video game music,Diamonds,Dirty Palm
1269,1269,0.4170,0.8810,5,-1.628,0,0.0749,0.000037,0.796000,0.237,...,audio_features,3n0zUFcqgG1Yg7BPJEC4uV,spotify:track:3n0zUFcqgG1Yg7BPJEC4uV,https://api.spotify.com/v1/tracks/3n0zUFcqgG1Y...,https://api.spotify.com/v1/audio-analysis/3n0z...,289434,4,Video game music,Colorblind,Panda Eyes
1270,1270,0.6360,0.5560,1,-8.046,0,0.1470,0.831000,0.000007,0.155,...,audio_features,4E7DSXRvLSi1E50k1JgSwO,spotify:track:4E7DSXRvLSi1E50k1JgSwO,https://api.spotify.com/v1/tracks/4E7DSXRvLSi1...,https://api.spotify.com/v1/audio-analysis/4E7D...,203595,4,Video game music,Bear Arms,chadster


In [48]:
df.to_csv(r"C:\Users\vikra\Documents\5153\spotifyGenresFinal.csv")